In [2]:
import random
import time
import math
import numpy as np
import copy
import json
import csv

In [31]:
def simulated_annealing(process, setups, loom_seq, assignments, Tcry, Tc, q, IT, lb, num_machines, num_orders):
    start_time = time.time()

    # calculate initial solution

    current_assignments = copy.deepcopy(assignments)
    current_seq = copy.deepcopy(loom_seq)
    current_obj, current_tables = sol_calc(setups, loom_seq, assignments, num_machines, num_orders)

    best_sol = {'Obj': current_obj, 'Seq': current_seq, 'Sol': current_assignments}
    interior_swap = 1
    while Tcry < Tc:
        for it in range(IT):
            # create new solution
            new_assignments = copy.deepcopy(current_assignments)
            new_seq = copy.deepcopy(current_seq)
            if interior_swap == 1:
                random_machine = random.randint(0, num_machines - 1)
                if len(new_seq[str(random_machine)]) > 3:
                    job1_pos = random.randint(1, len(loom_seq[str(random_machine)]) - 1)
                    job2_pos = random.choice(
                        list(range(1, job1_pos)) + list(range(job1_pos + 1, len(loom_seq[str(random_machine)]) - 1)))
                    new_seq[str(random_machine)][job1_pos], new_seq[str(random_machine)][job2_pos] = \
                    new_seq[str(random_machine)][job2_pos], new_seq[str(random_machine)][job1_pos]
                    interior_swap = 0
                elif len(new_seq[str(random_machine)]) > 2:
                    job1_pos = 1
                    job2_pos = 2
                    new_seq[str(random_machine)][job1_pos], new_seq[str(random_machine)][job2_pos] = \
                    new_seq[str(random_machine)][job2_pos], new_seq[str(random_machine)][job1_pos]
                    interior_swap = 0
                else:
                    interior_swap = 0
            else:
                if num_machines == 2:
                    random_machine_1 = 0
                    random_machine_2 = 1
                else:
                    random_machine_1 = random.randint(0, num_machines - 1)
                    random_machine_2 = random.choice(
                        list(range(0, random_machine_1)) + list(range(random_machine_1 + 1, num_machines - 1)))
                random_job_1 = random.choice(list(new_seq[str(random_machine_1)][1:]))
                job1_pos = new_seq[str(random_machine_1)].index(random_job_1)
                random_job_2 = random.choice(list(new_seq[str(random_machine_2)][1:]))
                job2_pos = new_seq[str(random_machine_2)].index(random_job_2)
                #print (new_seq)
                #print (random_job_1)
                #print (random_job_2)
                new_seq[str(random_machine_1)][job1_pos], new_seq[str(random_machine_2)][job2_pos] = \
                new_seq[str(random_machine_2)][job2_pos], new_seq[str(random_machine_1)][job1_pos]
                new_assignments[random_job_1, random_machine_2] = process[random_job_1,random_machine_1]
                new_assignments[random_job_1,random_machine_1] = 0.0
                new_assignments[random_job_2, random_machine_1] = process[random_job_2, random_machine_2]
                new_assignments[random_job_2, random_machine_2] = 0.0

                interior_swap = 1


            new_obj, new_tables = sol_calc(setups, new_seq, new_assignments, num_machines, num_orders)
            # print (current_obj, new_obj)
            try:
                Pacc = math.exp(round((current_obj - new_obj) / Tc, 2))
            except:
                Pacc = -1
            if best_sol['Obj'] > new_obj:
                best_sol['Obj'] = copy.deepcopy(new_obj)
                best_sol['Seq'] = copy.deepcopy(new_seq)
                best_sol['Sol'] = copy.deepcopy(new_assignments)

            if new_obj < current_obj or Pacc > random.random():
                current_obj = copy.deepcopy(new_obj)
                current_seq = copy.deepcopy(new_seq)
                current_assignments = copy.deepcopy(new_assignments)

        Tc = q * Tc

    sa_gap = round((best_sol['Obj'] - lb) / lb * 100,3)
    sa_time = round(time.time() - start_time,3)
    sa_sols = {'Time': sa_time, 'Gap': sa_gap, 'Obj': best_sol['Obj']}
    return sa_sols


def sol_calc(setups, loom_seq, assignments, num_machines, num_orders):
    tables = {}
    machine_load = {}
    for m in range(num_machines):  # number of machines
        machine_load[m] = 0
        tables[m] = np.full((num_orders + 1, 4), -1.0)
        for j in loom_seq[str(m)]:
            pos_j = loom_seq[str(m)].index(j)
            tables[m][j, 0] = machine_load[m]
            tables[m][j, 1] = float(setups[m][loom_seq[str(m)][pos_j - 1]][[loom_seq[str(m)][pos_j]]])
            tables[m][j, 2] = assignments[j, m]
            tables[m][j, 3] = tables[m][j, 0] + tables[m][j, 1] + tables[m][j, 2]
            machine_load[m] = tables[m][j, 3]
    #print (max(machine_load.values()))
    return max(machine_load.values()), tables

In [33]:
if __name__ == "__main__":
    columns = ['instance_number', 'instance', 'number_of_machines', 'number_of_orders', 'number_of_workers', 'alpha',
               'lower_bound', 'Tc', 'Tcry', 'q', 'IT', 'gha_gap', 'gha_obj', 'sa_Gap', 'sa_obj', 'sa_Time']
    with open('results.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(columns)
    input_data = open('N_10_20_30_40_50.json')
    data = json.load(input_data)
    # loop over all instances in json
    for instance_number in data.keys():
        # reading from instance json
        lower_bound = data[instance_number]['LB']
        number_of_orders = data[instance_number]['Number_of_jobs']
        number_of_machines = data[instance_number]['Number_of_machines']
        alpha = data[instance_number]['Alpha']
        process = np.array(list(data[instance_number]['ProcessTimes'].values())).T
        setups = {}
        for m in data[instance_number]['SetupTimes'].keys():
            setups[int(m)] = np.array(list(data[instance_number]['SetupTimes'][str(m)]))
        # loop for each worker
        for number_of_workers in [5]:
            if number_of_workers >= number_of_machines:
                instance = str(instance_number) + "-Work" + str(number_of_workers)
                input_sols = open('Final_solutions_10_20_30_40_50.json')
                sols = json.load(input_sols)
                initial_sols = sols[instance]
                print(initial_sols)
                lower_bound = initial_sols['LB']
                gha_obj = initial_sols['Total_Upper_Bound']
                gha_gap = initial_sols['Gap']
                loom_seq = sols[instance]['Loom_Sequence']
                assignments = np.array(list(sols[instance]['Assignments'].values())).T
                for Tc in [200, 500]:
                    for Tcry in [1, 20]:
                        for q in [0.90, 0.95]:
                            for IT in [50,100]:
                                for iterations in range(5):
                                    sa_sols = simulated_annealing(process, setups, loom_seq, assignments, Tcry,
                                                                                 Tc, q, IT, lower_bound, number_of_machines, number_of_orders)
                                    results = [instance_number, instance, number_of_machines, number_of_orders,
                                               number_of_workers, alpha,
                                               lower_bound, Tc, Tcry, q, IT, gha_gap, gha_obj, sa_sols['Gap'], sa_sols['Obj'],
                                               sa_sols['Time']]

                                    with open('results.csv', 'a', newline='') as file:
                                        writer = csv.writer(file)
                                        writer.writerow(results)

{'Number_of_jobs': 10, 'Number_of_machines': 2, 'Number_of_workers': 5, 'Alpha': 0, 'LB': 128.2802933480497, 'Loom_Sequence': {'0': [0, 8, 5, 3, 9, 6, 10], '1': [0, 1, 4, 2, 7]}, 'Assignments': {'0': [0.0, 0.0, 0.0, 10.46, 0.0, 27.82, 19.06, 0.0, 11.76, 17.28, 24.16], '1': [0.0, 70.23, 16.99, 0.0, 18.53, 0.0, 0.0, 25.26, 0.0, 0.0, 0.0]}, 'Master_Upper_Bound': 148.38, 'Total_Upper_Bound': 148.38, 'First_Step_Solution': 131.7768114360541, 'Gap': 15.668584883428538}
{'Number_of_jobs': 10, 'Number_of_machines': 2, 'Number_of_workers': 5, 'Alpha': 1, 'LB': 102.01458709379128, 'Loom_Sequence': {'0': [0, 2, 8, 3, 1, 10], '1': [0, 5, 6, 9, 7, 4, 10]}, 'Assignments': {'0': [0.0, 13.3, 23.83, 18.35, 0.0, 0.0, 0.0, 0.0, 15.71, 0.0, 29.76], '1': [0.0, 0.0, 0.0, 0.0, 5.66, 11.04, 16.37, 36.62, 0.0, 12.6, 19.05]}, 'Master_Upper_Bound': 103.57999999999998, 'Total_Upper_Bound': 103.58, 'First_Step_Solution': 102.64949214394322, 'Gap': 1.5344990856743794}
{'Number_of_jobs': 10, 'Number_of_machines': 2,

In [42]:
import pandas as pd

df = pd.read_csv('results.csv')
new_column_names = ['instance_number', 'instance', 'number_of_machines', 'number_of_orders','number_of_workers', 'alpha', 'lower_bound', 'Tc', 'Tcry', 'q', 'IT', 'gha_gap', 'gha_obj', 'sa_gap', 'sa_obj','sa_time']
df.columns = new_column_names
#df.head() #first 5 rows

In [43]:
#improved solutions with the SA
df_improved = df[df['gha_obj'] != df['sa_obj']]
df_improved

,instance_number,instance,number_of_machines,number_of_orders,number_of_workers,alpha,lower_bound,Tc,Tcry,q,IT,gha_gap,gha_obj,sa_gap,sa_obj,sa_time
1,1.639050e+09,1639049590.7328012-Work5,2,10,5,0,128.280293,200,1,0.90,50,15.668585,148.38,10.360,141.57,0.628
8,1.639050e+09,1639049590.7328012-Work5,2,10,5,0,128.280293,200,1,0.90,100,15.668585,148.38,14.944,147.45,0.566
9,1.639050e+09,1639049590.7328012-Work5,2,10,5,0,128.280293,200,1,0.90,100,15.668585,148.38,15.372,148.00,0.529
20,1.639050e+09,1639049590.7328012-Work5,2,10,5,0,128.280293,200,20,0.90,50,15.668585,148.38,12.948,144.89,0.113
21,1.639050e+09,1639049590.7328012-Work5,2,10,5,0,128.280293,200,20,0.90,50,15.668585,148.38,13.018,144.98,0.099
25,1.639050e+09,1639049590.7328012-Work5,2,10,5,0,128.280293,200,20,0.90,100,15.668585,148.38,15.404,148.04,0.201
30,1.639050e+09,1639049590.7328012-Work5,2,10,5,0,128.280293,200,20,0.95,50,15.668585,148.38,15.404,148.04,0.231
41,1.639050e+09,1639049590.7328012-Work5,2,10,5,0,128.280293,500,1,0.90,50,15.668585,148.38,10.438,141.67,0.263
50,1.639050e+09,1639049590.7328012-Work5,2,10,5,0,128.280293,500,1,0.95,50,15.668585,148.38,14.780,147.24,0.623
65,1.639050e+09,1639049590.7328012-Work5,2,10,5,0,128.280293,500,20,0.90,100,15.668585,148.38,13.548,145.66,0.552
